* カラムの特徴量から距離類似度の相関行列を求める

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import random, re

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

# データ読み込み

In [ ]:
df = pd.read_csv('../input/sample-data-wordanddoc2vec/actual_survey_data_jp.csv', index_col=0)
print(df.shape)
df.head()

## 算出した特徴ベクトルをロード

In [ ]:
'''行（ユーザー側）'''
df_user = pd.read_csv('../input/lowcol-actual-survey-data-fet12-randinit01/out_user.csv', index_col=0)
print(df_user.shape)
df_user.head()

In [ ]:
'''カラム側'''
df_col = pd.read_csv('../input/lowcol-actual-survey-data-fet12-randinit01/out_col.csv', index_col=0)
print(df_col.shape)
df_col.head()

In [ ]:
gamma = np.load('../input/lowcol-actual-survey-data-fet12-randinit01/out_gamma.npy').tolist()
gamma

# 距離類似度を算出
* 因子分析に入力するために変数（カラム側）の相関行列を求める
* ただし相関係数として、距離類似度を利用する
* 距離類似度は０〜１の範囲を取る

In [ ]:
'''距離類似度算出の関数を定義'''
def calc(target, df, gamma=1, sort=True):
    res = []
    d2 = ((df.values - target)**2).sum(axis=1)
    prob = np.exp(-gamma * d2)
    #print(prob)
    df_out = pd.DataFrame(df.index.values, index=df.index.values, columns=['title'])
    df_out['prob'] = prob
    if sort:
        df_out.sort_values('prob', inplace=True, ascending=False)
    return df_out

In [ ]:
target_col = df_col.index.values
target_col

In [ ]:
'''
「ブランド01」との相関（距離類似度）を計算

「ブランド01」と「ブランド02」は距離が近い
つまり、同時に出現しやすい（同時にonされやすい）
'''
calc(df_col.loc[target_col[0],:].values, df_col, gamma=gamma, sort=True)

In [ ]:
%%time
res_list = []
for idx in target_col:
    res = calc(df_col.loc[idx,:].values, df_col, gamma=gamma, sort=False)
    res2 = res
    res_list.append(res2['prob'].values)

In [ ]:
df_cor = pd.DataFrame(res_list, index=target_col, columns=target_col)
df_cor

In [ ]:
df_cor.to_csv('df_cor.csv')